In [1]:
import pandas as pd
from tqdm.autonotebook import tqdm
from lingua import Language, LanguageDetectorBuilder
import polars as pl

/scratch/slurm_tmpdir/job_23291498/ipykernel_516615/518065687.py:2: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### Imports

In [ ]:
# FILEPATH: /home/gomez/biobert-pytorch/snakemake_pmc/notebooks/analyze_sections.ipynb
df = pl.read_parquet('../pmc_tmp.parquet')

In [ ]:
df

In [5]:
df = df.filter(pl.col("text").str.len_chars() >= 40)

In [7]:
df = df.drop(['paragraph', 'sentence',"__index_level_0__"])


In [33]:
df = pl.read_parquet('temporary.parquet')

In [39]:
df = df.drop("english_confidence_values")

### Data

In [40]:
languages = [Language.ENGLISH, Language.FRENCH, Language.GERMAN, Language.SPANISH,Language.RUSSIAN,Language.CHINESE,Language.JAPANESE,Language.PORTUGUESE] 
detector = LanguageDetectorBuilder.from_languages(*languages).build()

In [41]:
confidences = []
texts = df["text"].to_list()
chunk_size = 2000000
for i in tqdm(range(0, len(texts), chunk_size)):
	chunk = texts[i:i+chunk_size]
	confidences.extend(detector.compute_language_confidence_values_in_parallel(chunk))


In [28]:
english_confidences = [x.value for sublist in confidences for x in sublist if x.language == Language.ENGLISH]

AttributeError: 'list' object has no attribute 'language'

In [ ]:
df.with_columns(pl.Series(name="english_confidence_values", values=english_confidences)) 

/tmp/ipykernel_235816/1433537886.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_small['english_confidence_values'] = [x[0].value for x in confidences]


In [ ]:
df = df.filter(pl.col("english_confidence_values") > 0.4)

In [5]:
df = df.drop("__index_level_0__")

In [8]:
df = df.filter(
	(pl.col("section").str.starts_with("Declaration") == False)
	& (pl.col("section").str.starts_with("Contribut") == False)
	& (pl.col("section").str.starts_with("Author") == False)
	& (pl.col("section").str.starts_with("AUTHOR") == False)
	& (pl.col("section").str.starts_with("Title") == False)
	& (pl.col("section").str.starts_with("Conflict") == False)
	& (pl.col("section").str.starts_with("Data sharing") == False)
	& (pl.col("section").str.starts_with("Competing") == False)
	& (pl.col("section").str.starts_with("CONFLICT") == False)
	& (pl.col("text").str.starts_with("- Reply") == False)
	& (pl.col("text").str.starts_with("- Line") == False)
	& (pl.col("section").str.contains("(?i)ethic") == False)
	& (pl.col("section").str.contains("(?i)contributions") == False)
	& (pl.col("section").str.contains("(?i)abbreviations") == False)
	& (pl.col("section").str.contains("(?i)Funding") == False)
	& (pl.col("section").str.contains("(?i)interest") == False)
	& (pl.col("section").str.contains("(?i)data availability") == False)
	& (pl.col("section").str.contains("(?i)acknowledgement") == False)
	& (pl.col("section").str.contains("(?i)ACKNOWLEDGMENT") == False)
	& (pl.col("section").str.contains("(?i)contribution statement") == False)
	& (pl.col("section").str.contains("(?i)Reviewers") == False)
	& (pl.col("section").str.contains("(?i)Family") == False)
	& (pl.col("section").str.contains("(?i)publisher") == False)
	& (pl.col("section").str.contains("(?i)Consent") == False)
	& (pl.col("section").str.contains("(?i)Disclosure") == False)
	& (pl.col("section").str.contains("病例与方法") == False)
	& (pl.col("section").str.contains("(?i)Hintergrund") == False)
	& (pl.col("section").str.contains("(?i)contributor") == False)
	& (pl.col("section").str.contains("(?i)credit author") == False)
	& (pl.col("section").str.contains("(?i)Software availability") == False)
)

In [ ]:
df.write_parquet('temporary_filtered.parquet')

In [2]:
df = pl.read_parquet('temporary_filtered.parquet')

In [14]:
df.filter(pl.col("section") == "Discusión")["text"][0]


'En diversos estudios clínicos y epidemiológicos, se ha determinado que las altas concentraciones de leptina en el suero se relacionan con un mayor riesgo de desarrollar cáncer de mama en mujeres antes y después de la menopausia ^27.'

In [22]:
detector.compute_language_confidence_values_in_parallel(["En diversos estudios clínicos y epidemiológicos, se ha determinado que las altas concentraciones de leptina en el suero se relacionan con un mayor riesgo de desarrollar cáncer de mama en mujeres antes y después de la menopausia ^27"],)

[[ConfidenceValue(language=Language.SPANISH, value=1),
  ConfidenceValue(language=Language.PORTUGUESE, value=0.00000000000000000009874631357461278),
  ConfidenceValue(language=Language.ENGLISH, value=0.000000000000000000000000000000005492987011162459),
  ConfidenceValue(language=Language.GERMAN, value=0.00000000000000000000000000000000000000000000000003361995537115124),
  ConfidenceValue(language=Language.FRENCH, value=0.00000000000000000000000000000000000000000000000000009797556881237097),
  ConfidenceValue(language=Language.CHINESE, value=0),
  ConfidenceValue(language=Language.JAPANESE, value=0),
  ConfidenceValue(language=Language.RUSSIAN, value=0)]]